In [12]:
import torch
import json
import cv2
import numpy as np
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")

Define dataset

In [14]:
class TennisCourtDataset(Dataset):
    def __init__(self, root_dir, data_file):
        self.root_dir = root_dir
        with open(data_file, "r") as f:
            self.data = json.load(f)

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        img = cv2.imread(f"{self.root_dir}/{item["id"]}.png")
        h, w = img.shape[:2]

        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transform(img)

        kps = np.array(item["kps"]).flatten()
        kps = kps.astype(np.float32)

        kps[::2] *= 224.0 / w
        kps[1::2] *= 224.0 / h

        return img, kps

Load dataset

In [15]:
train_dataset = TennisCourtDataset("data/images", "data/data_train.json")
val_dataset = TennisCourtDataset("data/images", "data/data_val.json")

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

Load model

In [16]:
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model.fc = torch.nn.Linear(model.fc.in_features, 14*2)
model = model.to(device)

Train

In [17]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 10
for epoch in range(epochs):
    for i, (imgs, kps) in enumerate(train_loader):
        imgs = imgs.to(device)
        kps = kps.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, kps)
        loss.backward()
        
        optimizer.step()
        if i%10==0:
            print(f"Epoch {epoch+1}, step {i+1} / {len(train_loader)}, loss = {loss.item()}")

Epoch 1, step 1 / 208, loss = 15187.0478515625
Epoch 1, step 11 / 208, loss = 13445.4072265625
Epoch 1, step 21 / 208, loss = 12046.6162109375
Epoch 1, step 31 / 208, loss = 10684.763671875
Epoch 1, step 41 / 208, loss = 9236.716796875
Epoch 1, step 51 / 208, loss = 7956.57275390625
Epoch 1, step 61 / 208, loss = 6933.89404296875
Epoch 1, step 71 / 208, loss = 5823.39111328125
Epoch 1, step 81 / 208, loss = 4823.42041015625


In [9]:
torch.save(model.state_dict(), "keypoint_model_resnet_18.pth")